### Imports:

In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from PIL import Image
import os
from tqdm import tqdm

import cv2
import skimage

from scipy.ndimage import binary_fill_holes
from skimage import img_as_ubyte
from skimage.color import rgb2gray, rgb2hsv
from skimage.exposure import equalize_hist, histogram
from skimage.feature import canny
from skimage.filters import threshold_otsu, unsharp_mask, gaussian
from skimage.measure import find_contours, approximate_polygon
from skimage.morphology import (dilation, erosion, binary_closing, binary_opening, remove_small_holes,
                                remove_small_objects, thin, convex_hull_image, square, disk, diamond, octagon, star, skeletonize)
from skimage.util import invert

from skimage.measure import label, regionprops

from scipy.signal import argrelmin, argrelmax, argrelextrema
from scipy.signal import find_peaks, find_peaks_cwt
from scipy.spatial import distance_matrix

In [2]:
def to_1ch(img, mode='gray'):
    transf_1ch = (
        {'r': lambda img: img[:, :, 0],
         'g': lambda img: img[:, :, 1],
         'b': lambda img: img[:, :, 2],
         'h': lambda img: img_as_ubyte(rgb2hsv(img)[:, :, 0]),
         's': lambda img: img_as_ubyte(rgb2hsv(img)[:, :, 1]),
         'v': lambda img: img_as_ubyte(rgb2hsv(img)[:, :, 2]),
         'gray': lambda img: img_as_ubyte(rgb2gray(img))}
    )
    return transf_1ch[mode](img)

In [3]:
def to_bin(img, thr='otsu'):
    if thr == 'otsu':
        return img > threshold_otsu(img)
    else:
        if thr < 1:
            return img > 255 * thr
        else:
            return img > thr

In [4]:
def to_uint8(img):
    img = img.astype(np.float)
    max_ = img.max()
    min_ = img.min()
    if min_ == max_:
        # constant image
        if max_:
            img *= 255 / max_
    elif min_ != 0 or max_ != 255:
        img = (img - min_) / (max_ - min_) * 255
    return img.astype(np.uint8)

In [5]:
def set_padding(img, num, val=False):
    if num > 0:
        h, w = img.shape
        new_img = np.full((h + 2 * num, w + 2 * num), val)
        new_img[num:-num, num:-num] = img
        img = new_img.astype(img.dtype)
    elif num < 0:
        num = -num
        img = img[num:-num, num:-num]
    return img

In [6]:
def resize_prop(img, max_size):
    h, w = img.shape[:2]
    img_max_size = max(w, h)
    if max_size < img_max_size:
        ratio = img_max_size / max_size
        w, h = int(w / ratio), int(h / ratio)
        
        img = cv2.resize(img, (w, h),
                         interpolation=cv2.INTER_AREA)
    return img

In [7]:
def img_sub(img1, img2):
    src_dtype = img1.dtype
    img1, img2 = list(map(lambda x: x.astype(np.int32), [img1, img2]))

    img = img1 - img2
    img[img < 0] = 0

    img = img.astype(src_dtype)
    return img

In [8]:
def sharpen(img, size, count):
    img_blur = cv2.GaussianBlur(img, (size,size), 0)
    img = cv2.addWeighted(img, count + 1, img_blur, -count, 0)
    return img

In [9]:
def list_get(list_, ind_, default_):
  try:
    return list_[ind_]
  except IndexError:
    return default_

def show_images(img_arr, name_arr=[], n_in_row=3, factor_r=8, factor_c=8, figsize=None,
         title=None, cmap_='Greys', label_fontsize=12, tick_fontsize=11, save_path=None):
    if type(img_arr) is np.ndarray:
        quant = img_arr.shape[0]
    elif type(img_arr) is list:
        quant = len(img_arr)
    cols = min(quant, n_in_row)
    rows = quant // n_in_row + (1 if quant % n_in_row else 0)
    if figsize is None:
        figsize = (int(factor_c * cols), int(factor_r * rows))
    fig = plt.figure(figsize=figsize)
    if title is not None:
        fig.suptitle(title, fontsize=14, y = 0.88)
    for i, img in enumerate(img_arr):
        ax = fig.add_subplot(rows, cols, i + 1)
        if type(img) is tuple:
            x, y = img
            ax.plot(x, y)
            ax.set_xlabel(list_get(name_arr, i, ''), fontsize=label_fontsize)
        else:
            ax.imshow(img, cmap=cmap_)
#             print(name_arr[i], len(img_arr), len(name_arr))
            ax.set_xlabel(list_get(name_arr, i, ''), fontsize=label_fontsize)
            ax.xaxis.set_major_locator(MaxNLocator(integer=True))
            ax.yaxis.set_major_locator(MaxNLocator(integer=True))
            ax.xaxis.set_ticks_position('top')
            ax.tick_params(axis='both', labelsize=tick_fontsize)
    plt.tight_layout()
    if save_path is not None:
        plt.savefig(f'{save_path}')
    plt.show()

In [10]:
def show_images_cv(img_arr, name_arr=[]):
    for i, img in enumerate(img_arr):
        if len(img.shape) == 3:
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        img = resize_prop(img, 700)
        cv2.imshow(list_get(name_arr, i, f'{i}'), img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [11]:
def apply_text(img, fig_text, coords, delta=(0,0), padding=(2, 3), bg_color=(255, 255, 255), font_scale=1.0, font=cv2.FONT_HERSHEY_DUPLEX):
    text_width, text_height = cv2.getTextSize(fig_text, font, fontScale=font_scale, thickness=1)[0]
    text_offset_x, text_offset_y = coords
    text_offset_x += delta[0]
    text_offset_y += delta[1]
    box_coords = ((text_offset_x - padding[0],
                   text_offset_y + padding[1]),
                  (text_offset_x + text_width + padding[0],
                   text_offset_y - text_height - padding[1]))
    cv2.rectangle(img, box_coords[0], box_coords[1], bg_color, cv2.FILLED)
    cv2.putText(img, fig_text, (text_offset_x, text_offset_y),
                font, fontScale=font_scale, color=(0, 0, 0), thickness=1)

In [12]:
def get_dist(p1, p2):
    p1, p2 = np.array(p1), np.array(p2)
    return np.sqrt(np.sum((p1 - p2)**2, axis=-1))

In [13]:
def get_angle(p1, p2, p3):
    a, b, c = (np.sqrt(np.sum((p3 - p1) ** 2, axis=-1)),
               np.sqrt(np.sum((p2 - p1) ** 2, axis=-1)),
               np.sqrt(np.sum((p2 - p3) ** 2, axis=-1)))
    return np.arccos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c))

### Finding hand mask

In [14]:
def descend_bin(img, factor=0.5, thr_a=0, thr_b=255, add_a=0, add_b=0, vis=False):
    y, x = list(map(np.array, histogram(img)))
    
    y_mean = np.mean(y)
    y_med = np.median(y)
    thr = y_med + np.abs(y_mean - y_med) * factor

    i0 = np.argmax(y)  
    i1, i2 = i0, i0
    i_min, i_max = 0, thr_b - thr_a

    for i in range(i0, i_min - 1, -1):
        if not (y[i] > thr):
            i1 = i
            break
        
    for i in range(i0, i_max + 1):
        if not (y[i] > thr):
            i2 = i
            break
            
    x1, x2 = x[i1], x[i2]
    
    x1 += add_a
    if x1 > 255:
        x1 = 255
    elif x1 < 0:
        x1 == 0

    x2 += add_b
    if x2 > 255:
        x2 = 255
    elif x2 < 0:
        x2 == 0

    img = img >= x2
    
    if vis:
        fig = plt.figure(figsize=(8, 8))
        plt.plot(x, y, label='y, histogram')

        y_mean_arr = np.full_like(y, y_mean)
        y_med_arr = np.full_like(y, y_med)
        thr_arr = np.full_like(y, thr)

        plt.plot(x, y_mean_arr, label='y mean')
        plt.plot(x, y_med_arr, label='y median')
        plt.plot(x, thr_arr, label='threshold')

        plt.plot(x1, y[i1], 'bo', label='left bound')
        plt.plot(x2, y[i2], 'ro', label='right bound')

        plt.legend()
        plt.tick_params(axis='both')
        plt.tight_layout()
    return img

In [15]:
def smart_bin(img):
    img = np.copy(img)

    mask = descend_bin(img, factor=0.0) > 0

    mask = set_padding(mask, 50, False)
    mask = remove_small_objects(mask, 10000, connectivity=2)
    mask = set_padding(mask, -50)
    return mask

def get_main_mask(img):
    # bluring
    img = cv2.GaussianBlur(img, (5,5), 0)
    
    # removing edges
    v, s, w = 2, 3, 3
    grad_x = cv2.Sobel(img, cv2.CV_64F, v, 0, ksize=s)
    grad_y = cv2.Sobel(img, cv2.CV_64F, 0, v, ksize=s)
    abs_grad_x = cv2.convertScaleAbs(grad_x)
    abs_grad_y = cv2.convertScaleAbs(grad_y)
    edges = cv2.addWeighted(abs_grad_x, w, abs_grad_y, w, 0)
    edges = edges * to_bin(edges, 60)
    edges = sharpen(edges, 3, 2.5)
    img = img_sub(img, edges)
    
    # binarization
    mask = smart_bin(img)
    
    # afterprocessing mask
    mask = binary_fill_holes(mask, disk(1))
    mask = remove_small_objects(mask, 100)

    return mask

def get_clean_mask_1(img):
    # binarization
    mask = smart_bin(img)

    # cleaning mask
    mask = thin(mask, 4)
    mask = binary_opening(mask, disk(5))

    # enlarging mask
    mask = dilation(mask, disk(3))
    mask = binary_fill_holes(mask, disk(1))
    mask = binary_closing(mask, disk(3))
    mask = binary_fill_holes(mask, disk(1))

    return mask

def get_clean_mask_2(img):
    # bluring
    img = cv2.GaussianBlur(img, (7, 7), 0)
    
    # binarization
    mask = smart_bin(img)
    
    # thinning
    mask = thin(mask, 2)

    return mask

def get_hand_mask(img):
    # get grayscaled image
    red = to_1ch(img, 'r')
    
    # get image masks
    main_mask = get_main_mask(red) > 0
    mask1 = get_clean_mask_1(red) > 0
    mask2 = get_clean_mask_2(red) > 0
    
    # masks logical AND
    res = main_mask * mask1 * mask2
    
    # afterprocessing mask
    res = binary_opening(res, disk(1))
    res = remove_small_objects(res, 500)
    res = binary_fill_holes(res, disk(1))
    
    return res

### Solution with convex defects

In [16]:
def get_point_ind(arr, point):
    arr = sorted(np.argwhere(arr == point), key=lambda x: x[0])
    l = len(arr)
    for i in range(l - 1):
        if arr[i][0] == arr[i+1][0]:
            return arr[i][0]
    return None

def get_points(img, mask, angle, vis=False):
    img = np.copy(img)
    mask = to_uint8(mask)
    n_tips = 5
    n_valleys = 4

    # construction of a contour and a convex hull with convex defects
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2:]
    contour = contours[np.argmax([len(contour) for contour in contours])][:, 0, :]
    hull = cv2.convexHull(contour)
    hull_inds = cv2.convexHull(contour, returnPoints=False)
    defects = cv2.convexityDefects(contour, hull_inds)

    # filtering convex defects by angle
    points_arr = []
    for i in range(defects.shape[0]):
        s,e,f,d = defects[i,0]
        start, end, far = contour[s], contour[e], contour[f]
        if get_angle(start, far, end) <= angle:
            points_arr.append([d, start, end, far])
    
    # filtering convex defects by depth
    points_arr = sorted(points_arr, key=lambda x: x[0])[-4:]
    
    # sorting convex defects in order of contour
    points_arr = sorted(points_arr, key=lambda x:
                        get_point_ind(contour, np.array(x[1])))

    
    # evaluating tips and collecting valleys
    points_arr = np.array([points[1:] for points in points_arr])
    start_arr, end_arr = points_arr[:, 0, :], points_arr[:, 1, :]
    dist_arr = [get_dist(start_arr[i], end_arr[(i - 1) % n_valleys])
                for i in range(n_valleys)]
    farest_tips_ind = np.argmax(dist_arr)
    tips, valleys = [0] * n_tips, points_arr[:, 2, :]
    j = 0
    for i in range(n_valleys):
        cur_start, prev_end = start_arr[i], end_arr[(i - 1) % n_valleys]
        if i == farest_tips_ind:
            if i == 0:
                tips[-1] = prev_end
            else:
                tips[j] = prev_end
                j += 1
            tips[j] = cur_start
            j += 1
        else:
            tips[j] = np.mean([cur_start, prev_end], axis=0).astype(np.int32)
            j += 1

    # finding max finger-line 'valley-tip' for palm orientation
    line_lens = []
    for i in range(n_valleys):
        j = i + 1 if farest_tips_ind > 0 and i >= farest_tips_ind else i
        v_point, t_point_1, t_point_2 = (valleys[i], tuple(tips[j]),
                                         tuple(tips[(j + 1) % n_tips]))
        line_lens.append([get_dist(v_point, t_point_1), get_dist(v_point, t_point_2)])
        cv2.line(img, tuple(v_point), tuple(t_point_1), [0,255,0], 2)
        cv2.line(img, tuple(v_point), tuple(t_point_2), [255,0,255], 2)
    max_line_ind = np.argmax(np.array(line_lens).ravel())
    
    # palm orientation
    palm_orient = 'r' if max_line_ind % 2 else 'l'
    first_ind = max_line_ind // 2

    
    # sorting tips and valleys in normal order
    t_inds, v_inds = list(range(n_tips)), list(range(n_valleys))
    if palm_orient == 'r':
        if first_ind != 0:
            v_inds = v_inds[first_ind:] + v_inds[:first_ind]
            t_inds = t_inds[first_ind + 1:] + t_inds[:first_ind + 1]
    elif palm_orient == 'l':
        if first_ind != 0:
            v_inds = v_inds[first_ind:] + v_inds[:first_ind]
            t_inds = t_inds[first_ind:] + t_inds[:first_ind]
        v_inds = [v_inds[(n_valleys - i) % n_valleys] for i in range(n_valleys)]
        t_inds = (t_inds[2:] + t_inds[:2])[::-1]
    else:
        raise RuntimeError(f'Wrong palm orientation {palm_orient}!')
    tips = np.array([tips[ind] for ind in t_inds])
    valleys = np.array([valleys[ind] for ind in v_inds])
    
    if vis:
        contour_color, hull_color = [255,255,0], [0,255,255]
        v_color, t_color, line_color = [255,0,0], [0,0,255], [0,255, 0]
        
        
        cv2.drawContours(img, [contour], 0, contour_color, 2)
        cv2.drawContours(img, [hull], 0, hull_color, 2)

        for i in range(n_valleys):
            v_point, t_point_1, t_point_2 = list(map(lambda x: tuple(x),
                                            [valleys[i], tips[i], tips[i + 1]]))

            cv2.line(img, v_point, t_point_1, line_color, 2)
            cv2.line(img, v_point, t_point_2, line_color, 2)
            
            cv2.circle(img, v_point, 6, v_color, -1)
            cv2.circle(img, t_point_1, 6, t_color, -1)
            cv2.circle(img, t_point_2, 6, t_color, -1)

        for i in range(n_valleys):
            apply_text(img, f'{i + 1}', tuple(valleys[i]), font_scale=0.7)

        for i in range(n_tips):
            apply_text(img, f'{i + 1}', tuple(tips[i]), font_scale=0.7)

    return img

In [17]:
# parameters
input_folder = 'Training'
output_folder = 'out'

# input
name = input()
img = cv2.imread(f'{input_folder}/{name}')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# show input file
show_images_cv([img], [name])

# processing
mask = get_hand_mask(img)
img_res = get_points(img, mask, np.pi, True)

# save
try:
    Image.fromarray(img_res).save(f'{output_folder}/res_conv_{name}')
except:
    print('Can not save result!')

# show result file
show_images_cv([img_res], [f'{name} result'])

017.tif
